In [2]:
!wget https://uaelxg.bn.files.1drv.com/y4mpisY5MwS0-xepR1icDqLmai7cO4J4vlqRC4TpyiWqOoB_R2Q0sXFXwUUgWVJpdhrqJNFZX78DLsfMviiQY6jU08MEYeDU_rp4u5Jsb9ZZ35eFduGKzFEFGWlsN63KCJYwaRzim4Y9wM6mJtgAIAQ3j4bCZXbkPbignxOBkzD3xRjt5Tso4QOXxdiMK5DregI/fruits-360_dataset_2018_06_03.zip
  
 

--2018-06-21 02:43:06--  https://uaelxg.bn.files.1drv.com/y4mpisY5MwS0-xepR1icDqLmai7cO4J4vlqRC4TpyiWqOoB_R2Q0sXFXwUUgWVJpdhrqJNFZX78DLsfMviiQY6jU08MEYeDU_rp4u5Jsb9ZZ35eFduGKzFEFGWlsN63KCJYwaRzim4Y9wM6mJtgAIAQ3j4bCZXbkPbignxOBkzD3xRjt5Tso4QOXxdiMK5DregI/fruits-360_dataset_2018_06_03.zip
Resolving uaelxg.bn.files.1drv.com (uaelxg.bn.files.1drv.com)... 204.79.197.213
Connecting to uaelxg.bn.files.1drv.com (uaelxg.bn.files.1drv.com)|204.79.197.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211922343 (202M) [application/zip]
Saving to: ‘fruits-360_dataset_2018_06_03.zip’

fruits-360_dataset_ 100%[===================>] 202.10M  23.1MB/s    in 9.1s    

2018-06-21 02:43:17 (22.1 MB/s) - ‘fruits-360_dataset_2018_06_03.zip’ saved [211922343/211922343]



In [0]:
import zipfile
zip_ref = zipfile.ZipFile('fruits-360_dataset_2018_06_03.zip', 'r')
zip_ref.extractall('data2')
zip_ref.close()

In [1]:
ls

datalab/


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import glob
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [4]:
!ls


data2  datalab	fruits-360_dataset_2018_06_03.zip  log64  logs


In [0]:
training_fruit_img = []
training_label = []
for dir_path in glob.glob("data2/fruits-360/Training/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        training_fruit_img.append(image)
        training_label.append(img_label)
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)
label_to_id = {v:k for k,v in enumerate(np.unique(training_label)) }
id_to_label = {v:k for k,v in label_to_id.items()}
training_label_id = np.array([label_to_id[i] for i in training_label])
validation_fruit_img=[]
validation_label =[]
for dir_path in glob.glob("data2/fruits-360/Validation/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_fruit_img.append(image)
        validation_label.append(img_label)
validation_fruit_img = np.array(validation_fruit_img)
validation_label = np.array(validation_label)
validation_label_id = np.array([label_to_id[i] for i in validation_label])

In [23]:
training_fruit_img.shape

(32426, 64, 64, 3)

In [0]:
x_test = validation_fruit_img.astype('float32')
y_test = validation_label_id



def cnn(batch_size = 128, num_classes = 65, epochs = 1, numfilter1 = 64,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='logs', optimizer=keras.optimizers.Adadelta()):
  

  batch_size = batch_size
  num_classes = num_classes
  epochs = epochs

  # input image dimensions
  img_rows, img_cols = 64, 64
  input_shape = (img_rows,img_cols,3)


  y_train = training_label_id
  y_test = validation_label_id

  x_train = training_fruit_img.astype('float32')
  x_test = validation_fruit_img.astype('float32')
  x_train /= 255
  x_test /= 255
  print('x_train shape:', x_train.shape)
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')

  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)

  model = Sequential()
  
  model.add(Conv2D(numfilter1, kernel_size=kernel_size1,
                   activation=activation,
                   input_shape=input_shape))
  model.add(Conv2D(numfilter2, kernel_size2, activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(numfilter1, kernel_size=kernel_size1,
                   activation=activation,
                   input_shape=input_shape))
  model.add(Conv2D(numfilter2, kernel_size2, activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(128, activation=activation))
  model.add(Dense(num_classes, activation='softmax'))

  from keras.callbacks import TensorBoard

  tensorboard = TensorBoard(log_dir='logs', histogram_freq=0,
                          write_graph=True, write_images=False)
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard])
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  
  return model



In [4]:
model = cnn(batch_size = 128, num_classes = 65, epochs = 10, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='logs', optimizer=keras.optimizers.Adadelta())
from sklearn.metrics import classification_report
classes = np.argmax(model.predict(x_test, batch_size = 128), axis=1)
print(classification_report(y_test, classes))

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/10
32426/32426 [==============================] - 26s 805us/step - loss: 1.6910 - acc: 0.5696 - val_loss: 0.5614 - val_acc: 0.8685
Epoch 2/10
15872/32426 [=============>................] - ETA: 11s - loss: 0.1099 - acc: 0.9705

32426/32426 [==============================] - 24s 752us/step - loss: 0.0936 - acc: 0.9752 - val_loss: 0.2608 - val_acc: 0.9233
Epoch 3/10
32426/32426 [==============================] - 25s 756us/step - loss: 0.0285 - acc: 0.9892 - val_loss: 0.2137 - val_acc: 0.9328
Epoch 4/10
11392/32426 [=========>....................] - ETA: 14s - loss: 0.1950 - acc: 0.9596

32426/32426 [==============================] - 24s 754us/step - loss: 0.0789 - acc: 0.9809 - val_loss: 0.2675 - val_acc: 0.9301
Epoch 5/10
32426/32426 [==============================] - 24s 753us/step - loss: 0.0368 - acc: 0.9876 - val_loss: 0.3290 - val_acc: 0.9147
Epoch 6/10
10368/32426 [========>.....................] - ETA: 14s - loss: 0.0455 - acc: 0.9844

32426/32426 [==============================] - 24s 754us/step - loss: 0.0360 - acc: 0.9876 - val_loss: 0.2589 - val_acc: 0.9258
Epoch 7/10
32426/32426 [==============================] - 25s 757us/step - loss: 0.0150 - acc: 0.9917 - val_loss: 0.2442 - val_acc: 0.9308
Epoch 8/10
10112/32426 [========>.....................] - ETA: 15s - loss: 0.2138 - acc: 0.9539

32426/32426 [==============================] - 24s 756us/step - loss: 0.0762 - acc: 0.9802 - val_loss: 0.2756 - val_acc: 0.9317
Epoch 9/10
32426/32426 [==============================] - 25s 757us/step - loss: 0.0110 - acc: 0.9931 - val_loss: 0.2875 - val_acc: 0.9336
Epoch 10/10
10112/32426 [========>.....................] - ETA: 15s - loss: 0.0098 - acc: 0.9922

32426/32426 [==============================] - 24s 755us/step - loss: 0.0103 - acc: 0.9927 - val_loss: 0.2890 - val_acc: 0.9373
Test loss: 0.2889807147027679
Test accuracy: 0.9372649729432266
             precision    recall  f1-score   support

          0       1.00      0.10      0.19       164
          1       0.00      0.00      0.00       164
          2       0.24      0.29      0.26       164
          3       0.00      0.00      0.00       161
          4       1.00      0.04      0.07       164
          5       0.43      0.14      0.21       164
          6       0.29      0.21      0.24       164
          7       1.00      0.09      0.17       144
          8       1.00      0.58      0.73       166
          9       0.27      0.70      0.39       164
         10       0.71      1.00      0.83       164
         11       1.00      0.11      0.20       143
         12       0.50      0.01      0.01       166
         13       0.78      0.74      0.76       166
         14 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:

def cnn2(batch_size = 128, num_classes = 65, epochs = 30, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='tanh', optimizer=keras.optimizers.Adadelta(),pool_size=(2, 2) ):
  
  tensorboard = keras.callbacks.TensorBoard(log_dir='logrms', histogram_freq=0,
                          write_graph=True, write_images=False)
  batch_size = batch_size
  num_classes = num_classes
  epochs = epochs

  # input image dimensions
  img_rows, img_cols = 64, 64
  input_shape = (img_rows,img_cols,3)


  y_train = training_label_id
  y_test = validation_label_id

  x_train = training_fruit_img.astype('float32')
  x_test = validation_fruit_img.astype('float32')
  x_train /= 255
  x_test /= 255
  print('x_train shape:', x_train.shape)
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')

  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)

  model = Sequential()
  model.add(Conv2D(numfilter1, kernel_size=kernel_size1,
                   activation=activation,
                   input_shape=input_shape))
  model.add(Conv2D(numfilter2, kernel_size2, activation=activation))
  model.add(MaxPooling2D(pool_size=pool_size))
  model.add(Dropout(0.5))
  model.add(Conv2D(numfilter1, kernel_size=kernel_size1,
                   activation=activation,
                   input_shape=input_shape))
  model.add(Conv2D(numfilter2, kernel_size2, activation=activation))
  model.add(MaxPooling2D(pool_size=pool_size))
  model.add(Flatten())
  model.add(Dense(256, activation=activation))
  model.add(Dropout(0.25))
  model.add(Dense(num_classes, activation='softmax'))

  from keras.callbacks import TensorBoard

  tensorboard = TensorBoard(log_dir='logs2', histogram_freq=0,
                          write_graph=True, write_images=False)
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard])
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  return model

In [6]:
model2 = cnn2(batch_size = 80, num_classes = 65, epochs = 27, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', optimizer = keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0),pool_size=(2, 2))



x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/27
32426/32426 [==============================] - 29s 902us/step - loss: 1.1904 - acc: 0.6684 - val_loss: 0.5375 - val_acc: 0.8658
Epoch 2/27
 2320/32426 [=>............................] - ETA: 24s - loss: 0.2407 - acc: 0.9216

32426/32426 [==============================] - 28s 877us/step - loss: 0.1588 - acc: 0.9502 - val_loss: 0.5034 - val_acc: 0.8416
Epoch 3/27
23920/32426 [=====================>........] - ETA: 6s - loss: 0.0862 - acc: 0.9722

32426/32426 [==============================] - 28s 877us/step - loss: 0.0820 - acc: 0.9735 - val_loss: 0.2096 - val_acc: 0.9405
Epoch 4/27
31840/32426 [============================>.] - ETA: 0s - loss: 0.0526 - acc: 0.9818

32426/32426 [==============================] - 28s 879us/step - loss: 0.0526 - acc: 0.9817 - val_loss: 0.3462 - val_acc: 0.9077
Epoch 5/27
32426/32426 [==============================] - 28s 876us/step - loss: 0.0432 - acc: 0.9846 - val_loss: 0.1572 - val_acc: 0.9554
Epoch 6/27
 1600/32426 [>.............................] - ETA: 24s - loss: 0.0204 - acc: 0.9913

32426/32426 [==============================] - 28s 873us/step - loss: 0.0353 - acc: 0.9868 - val_loss: 0.1559 - val_acc: 0.9606
Epoch 7/27
24240/32426 [=====================>........] - ETA: 6s - loss: 0.0315 - acc: 0.9876

...

32426/32426 [==============================] - 28s 877us/step - loss: 0.0186 - acc: 0.9911 - val_loss: 0.2521 - val_acc: 0.9615
Epoch 23/27
32240/32426 [============================>.] - ETA: 0s - loss: 0.0216 - acc: 0.9908

32426/32426 [==============================] - 28s 875us/step - loss: 0.0216 - acc: 0.9908 - val_loss: 0.2019 - val_acc: 0.9621
Epoch 24/27
32426/32426 [==============================] - 28s 874us/step - loss: 0.0238 - acc: 0.9909 - val_loss: 0.3348 - val_acc: 0.9512
Epoch 25/27
 1840/32426 [>.............................] - ETA: 23s - loss: 0.0221 - acc: 0.9929

32426/32426 [==============================] - 28s 876us/step - loss: 0.0188 - acc: 0.9917 - val_loss: 0.2423 - val_acc: 0.9589
Epoch 26/27
23920/32426 [=====================>........] - ETA: 6s - loss: 0.0220 - acc: 0.9917

32426/32426 [==============================] - 28s 875us/step - loss: 0.0202 - acc: 0.9916 - val_loss: 0.3202 - val_acc: 0.9586
Epoch 27/27
32426/32426 [==============================] - 28s 873us/step - loss: 0.0201 - acc: 0.9911 - val_loss: 0.2627 - val_acc: 0.9654


Test loss: 0.26274687034063077
Test accuracy: 0.9654223608181235


In [12]:
from sklearn.metrics import classification_report
classes = np.argmax(model2.predict(x_test, batch_size=128), axis=1)
print(classification_report(y_test, classes))

             precision    recall  f1-score   support

          0       0.95      0.46      0.62       164
          1       0.20      0.93      0.33       164
          2       0.94      0.30      0.45       164
          3       0.00      0.00      0.00       161
          4       0.00      0.00      0.00       164
          5       0.49      0.60      0.54       164
          6       1.00      0.07      0.13       164
          7       1.00      0.10      0.18       144
          8       1.00      0.21      0.35       166
          9       0.24      1.00      0.39       164
         10       0.98      1.00      0.99       164
         11       0.00      0.00      0.00       143
         12       1.00      0.54      0.70       166
         13       1.00      0.75      0.86       166
         14       0.77      0.86      0.81       166
         15       0.86      0.80      0.83       166
         16       0.98      1.00      0.99       164
         17       0.80      1.00      0.89   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
np.sum(y_test==classes)

7498

In [6]:
model3 = cnn2(batch_size = 80, num_classes = 65, epochs = 27, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', optimizer = keras.optimizers.SGD(),pool_size=(2, 2))

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/27
32426/32426 [==============================] - 28s 877us/step - loss: 3.3952 - acc: 0.1512 - val_loss: 2.4215 - val_acc: 0.3579
Epoch 2/27
 2480/32426 [=>............................] - ETA: 22s - loss: 2.1255 - acc: 0.4000

32426/32426 [==============================] - 27s 836us/step - loss: 1.3927 - acc: 0.5915 - val_loss: 1.4043 - val_acc: 0.6249
Epoch 3/27
24560/32426 [=====================>........] - ETA: 5s - loss: 0.6764 - acc: 0.7928

32426/32426 [==============================] - 27s 838us/step - loss: 0.6138 - acc: 0.8112 - val_loss: 1.6041 - val_acc: 0.6032
Epoch 4/27
32426/32426 [==============================] - 27s 838us/step - loss: 0.3207 - acc: 0.8976 - val_loss: 0.6570 - val_acc: 0.7989
Epoch 5/27
  240/32426 [..............................] - ETA: 24s - loss: 0.2920 - acc: 0.8958

32426/32426 [==============================] - 27s 839us/step - loss: 0.1794 - acc: 0.9419 - val_loss: 0.5948 - val_acc: 0.8459
Epoch 6/27
23760/32426 [====================>.........] - ETA: 6s - loss: 0.1290 - acc: 0.9579

32426/32426 [==============================] - 27s 836us/step - loss: 0.1219 - acc: 0.9600 - val_loss: 0.4855 - val_acc: 0.8761
Epoch 7/27
32426/32426 [==============================] - 27s 836us/step - loss: 0.0916 - acc: 0.9688 - val_loss: 0.4472 - val_acc: 0.8856
Epoch 8/27
   80/32426 [..............................] - ETA: 24s - loss: 0.1046 - acc: 0.9625

...

32426/32426 [==============================] - 27s 840us/step - loss: 0.0228 - acc: 0.9892 - val_loss: 0.3518 - val_acc: 0.9241
Epoch 23/27
23520/32426 [====================>.........] - ETA: 6s - loss: 0.0198 - acc: 0.9909

32426/32426 [==============================] - 27s 841us/step - loss: 0.0199 - acc: 0.9904 - val_loss: 0.3434 - val_acc: 0.9304
Epoch 24/27
32400/32426 [============================>.] - ETA: 0s - loss: 0.0223 - acc: 0.9896

32426/32426 [==============================] - 27s 837us/step - loss: 0.0222 - acc: 0.9896 - val_loss: 0.3295 - val_acc: 0.9296
Epoch 25/27
32426/32426 [==============================] - 27s 839us/step - loss: 0.0198 - acc: 0.9901 - val_loss: 0.3776 - val_acc: 0.9203
Epoch 26/27
 1920/32426 [>.............................] - ETA: 23s - loss: 0.0247 - acc: 0.9885

32426/32426 [==============================] - 27s 840us/step - loss: 0.0212 - acc: 0.9902 - val_loss: 0.3963 - val_acc: 0.9220
Epoch 27/27
24240/32426 [=====================>........] - ETA: 6s - loss: 0.0186 - acc: 0.9904

32426/32426 [==============================] - 27s 841us/step - loss: 0.0178 - acc: 0.9907 - val_loss: 0.3629 - val_acc: 0.9281
Test loss: 0.362897037049418
Test accuracy: 0.928093185361827


In [9]:
from sklearn.metrics import classification_report
classes = np.argmax(model3.predict(x_test, batch_size=128), axis=1)
print(classification_report(y_test, classes))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00       164
          1       0.03      0.09      0.05       164
          2       0.04      0.07      0.05       164
          3       0.00      0.00      0.00       161
          4       0.00      0.00      0.00       164
          5       0.00      0.00      0.00       164
          6       0.00      0.00      0.00       164
          7       0.00      0.00      0.00       144
          8       1.00      0.03      0.06       166
          9       0.33      0.46      0.38       164
         10       0.00      0.00      0.00       164
         11       1.00      0.02      0.04       143
         12       0.00      0.00      0.00       166
         13       0.25      0.26      0.26       166
         14       0.86      0.19      0.32       166
         15       0.17      0.51      0.26       166
         16       0.08      1.00      0.14       164
         17       0.07      0.40      0.12   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:

cnn2(batch_size = 256, num_classes = 65, epochs = 50, numfilter1 = 32,
        kernel_size1=(3, 3), numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='log64')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/50
32426/32426 [==============================] - 23s 710us/step - loss: 3.1577 - acc: 0.1975 - val_loss: 1.7576 - val_acc: 0.5215
Epoch 2/50
32426/32426 [==============================] - 22s 675us/step - loss: 1.3814 - acc: 0.6004 - val_loss: 0.8117 - val_acc: 0.7871
Epoch 3/50
13824/32426 [===========>..................] - ETA: 11s - loss: 0.7788 - acc: 0.7684

32426/32426 [==============================] - 22s 677us/step - loss: 0.6457 - acc: 0.8081 - val_loss: 0.5618 - val_acc: 0.8385
Epoch 4/50
32426/32426 [==============================] - 22s 689us/step - loss: 0.3510 - acc: 0.8959 - val_loss: 0.3707 - val_acc: 0.8892
Epoch 5/50
32426/32426 [==============================] - 22s 691us/step - loss: 0.2277 - acc: 0.9321 - val_loss: 0.3485 - val_acc: 0.8897
Epoch 6/50
 9984/32426 [========>.....................] - ETA: 13s - loss: 0.1753 - acc: 0.9493

32426/32426 [==============================] - 22s 682us/step - loss: 0.1661 - acc: 0.9506 - val_loss: 0.2608 - val_acc: 0.9147
Epoch 7/50
32426/32426 [==============================] - 22s 675us/step - loss: 0.1267 - acc: 0.9629 - val_loss: 0.2251 - val_acc: 0.9297
Epoch 8/50
32426/32426 [==============================] - 22s 694us/step - loss: 0.0980 - acc: 0.9714 - val_loss: 0.2699 - val_acc: 0.9182
Epoch 9/50
 9216/32426 [=======>......................] - ETA: 14s - loss: 0.0894 - acc: 0.9729

32426/32426 [==============================] - 23s 697us/step - loss: 0.0821 - acc: 0.9759 - val_loss: 0.2050 - val_acc: 0.9341
Epoch 10/50
32426/32426 [==============================] - 22s 684us/step - loss: 0.0637 - acc: 0.9814 - val_loss: 0.1947 - val_acc: 0.9332
Epoch 11/50
32426/32426 [==============================] - 22s 678us/step - loss: 0.0576 - acc: 0.9833 - val_loss: 0.1810 - val_acc: 0.9423
Epoch 12/50
 8960/32426 [=======>......................] - ETA: 14s - loss: 0.0541 - acc: 0.9838

32426/32426 [==============================] - 22s 679us/step - loss: 0.0538 - acc: 0.9830 - val_loss: 0.1717 - val_acc: 0.9417
Epoch 13/50
32426/32426 [==============================] - 22s 686us/step - loss: 0.0506 - acc: 0.9845 - val_loss: 0.1879 - val_acc: 0.9374
Epoch 14/50
32426/32426 [==============================] - 22s 693us/step - loss: 0.0476 - acc: 0.9846 - val_loss: 0.1782 - val_acc: 0.9388
Epoch 15/50
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0438 - acc: 0.9855

...

32426/32426 [==============================] - 22s 681us/step - loss: 0.0260 - acc: 0.9897 - val_loss: 0.1553 - val_acc: 0.9455
Epoch 37/50
32426/32426 [==============================] - 22s 681us/step - loss: 0.0253 - acc: 0.9900 - val_loss: 0.1611 - val_acc: 0.9441
Epoch 38/50
32426/32426 [==============================] - 22s 690us/step - loss: 0.0244 - acc: 0.9903 - val_loss: 0.1541 - val_acc: 0.9463
Epoch 39/50
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0255 - acc: 0.9901

32426/32426 [==============================] - 23s 701us/step - loss: 0.0250 - acc: 0.9901 - val_loss: 0.1561 - val_acc: 0.9467
Epoch 40/50
32426/32426 [==============================] - 22s 687us/step - loss: 0.0259 - acc: 0.9897 - val_loss: 0.1554 - val_acc: 0.9463
Epoch 41/50
32426/32426 [==============================] - 22s 677us/step - loss: 0.0253 - acc: 0.9899 - val_loss: 0.1601 - val_acc: 0.9454
Epoch 42/50
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0228 - acc: 0.9906

32426/32426 [==============================] - 22s 677us/step - loss: 0.0251 - acc: 0.9895 - val_loss: 0.1550 - val_acc: 0.9469
Epoch 43/50
32426/32426 [==============================] - 22s 685us/step - loss: 0.0244 - acc: 0.9912 - val_loss: 0.1574 - val_acc: 0.9469
Epoch 44/50
32426/32426 [==============================] - 22s 685us/step - loss: 0.0245 - acc: 0.9902 - val_loss: 0.1556 - val_acc: 0.9481
Epoch 45/50
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0248 - acc: 0.9892

32426/32426 [==============================] - 22s 686us/step - loss: 0.0249 - acc: 0.9897 - val_loss: 0.1536 - val_acc: 0.9480
Epoch 46/50
32426/32426 [==============================] - 22s 676us/step - loss: 0.0234 - acc: 0.9908 - val_loss: 0.1524 - val_acc: 0.9491
Epoch 47/50
32426/32426 [==============================] - 22s 684us/step - loss: 0.0249 - acc: 0.9908 - val_loss: 0.1574 - val_acc: 0.9472
Epoch 48/50
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0226 - acc: 0.9905

32426/32426 [==============================] - 23s 695us/step - loss: 0.0243 - acc: 0.9906 - val_loss: 0.1568 - val_acc: 0.9474
Epoch 49/50
32426/32426 [==============================] - 22s 693us/step - loss: 0.0233 - acc: 0.9908 - val_loss: 0.1587 - val_acc: 0.9475
Epoch 50/50
32426/32426 [==============================] - 22s 683us/step - loss: 0.0241 - acc: 0.9900 - val_loss: 0.1548 - val_acc: 0.9477
Test loss: 0.15476946327961885
Test accuracy: 0.9477208107860222


In [9]:
!ls

data2	 fruits-360_dataset_2018_06_03.zip  logs   logs2.zip.zip
datalab  logrms2.zip.zip		    logs2


In [7]:
import shutil
shutil.make_archive('logs2.zip', 'zip', 'logs2')

'/content/logs2.zip.zip'

In [8]:
from google.colab import files
files.download('/content/logs2.zip.zip')  # from colab to browser download

In [0]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
pca = PCA()
pca_result2 = pca.fit(validation_fruit_img.reshape(validation_fruit_img.shape[0],64*64*3))


In [6]:

rf = RandomForestClassifier(n_estimators=100)
rf.fit(pca_result1, training_label_id)
pred = rf.predict(pca_result2)


from sklearn.metrics import accuracy_score

print(accuracy_score(validation_label_id, pred))

0.5173805374667523
